In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from sqlalchemy import create_engine
from splinter import Browser

In [63]:
#splinter
browser = Browser('chrome', headless=False)
main_url='https://apps.oaklandca.gov/pec/Lobbyist_ReportsFiled.aspx'
browser.visit(main_url)

The chromedriver version (115.0.5790.170) detected in PATH at C:\utilities\chromedriver.exe might not be compatible with the detected chrome version (116.0.5845.188); currently, chromedriver 116.0.5845.96 is recommended for chrome 116.*, so it is advised to delete the driver in PATH and retry


In [64]:
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [65]:
records=[]
tds=soup.find_all('td',class_=['TableRecords_OddLine','TableRecords_EvenLine'])
links = [td.find('a')['href'] for td in tds if td.find('a')]

In [66]:
def navigation_has_loaded():
    if browser.is_element_present_by_css('.ListNavigation_Next', wait_time=.000001): # Check for next in case on first page
        return True
    elif browser.is_element_present_by_css('.ListNavigation_Previous', wait_time=.000001): # Check for previous in case on last page
        return True
    else: # Navigation has not loaded
        return False

In [67]:
navigation_has_loaded()

True

In [68]:
def contact_scrape():
    filing_html=browser.html
    filing_soup=BeautifulSoup(filing_html,'html.parser')
    tds=filing_soup.find_all('tr')
    for td in tds:
        # Check if this row contains 'Nothing reported...'
        if td.find(string="Nothing reported..."):
            continue  # Skip this iteration if found
            # Convert td elements to text and store them in the 'columns' list
        columns = [col.get_text(strip=True) for col in td]
        # You can add validation here to check if the length of columns is as expected
        # For example, if you expect each row to have 8 columns, you can check:
        if len(columns) == 8:
            records.append(columns)
        else:
            print(f"Skipped row with {len(columns)} columns: {columns}")

In [69]:
def lobbyist_scrape(arg):
  for link in arg:
      browser.visit('https://apps.oaklandca.gov/pec/'+link)
      if navigation_has_loaded():
        while browser.is_element_present_by_css('.ListNavigation_Previous', wait_time=.000001):
          try:
            previous_button=browser.find_by_css('.ListNavigation_Previous')
            previous_button.click()
          except Exception as e:
            print('Reset pages')
            break
        while navigation_has_loaded():
          contact_scrape()
          try:
            next_button = browser.find_by_css('.ListNavigation_Next')
            next_button.click()
            time.sleep(random.uniform(.5, 1))
          except Exception as e: #ends loop on last page
            print(f"Error or end of pages: {e}")
            break
      else:
         contact_scrape()
      if navigation_has_loaded():
        try:
          page_1_button = browser.find_by_xpath('//*[@id="wt133_OutSystemsUIWeb_wt2_block_wtContent_wtMainContent_OutSystemsUIWeb_wt223_block_wtContent_OutSystemsUIWeb_wtSchAAcc_block_wtContent_RichWidgets_wt5_block_wtPageNavigator_ctl00_wtLink8"]')
          page_1_button.click()
        except Exception as e:
          print(f"Error or end of pages: {e}")

In [70]:
#test
browser.visit(main_url)
records=[]
#lobbyist_scrape()

In [71]:
def test(arg):
    records=[]
    x=0
    while x<arg:
        if navigation_has_loaded():
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            tds=soup.find_all('td',class_=['TableRecords_OddLine','TableRecords_EvenLine'])
            links = [td.find('a')['href'] for td in tds if td.find('a')]
            lobbyist_scrape()
            browser.visit(main_url)
            if navigation_has_loaded():
                try:
                    next_button = browser.find_by_css('.ListNavigation_Next')
                    next_button.click()
                    time.sleep(random.uniform(1, 1.1))
                except Exception as e: #ends loop on last page
                    print(f"Error or end of pages: {e}")
                    break
            else:
                print('bruh 2')
                break
        else:
            print('bruh')
            break
        x+=1

In [72]:
def full_scrape():
    while True:
        if navigation_has_loaded():
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            tds=soup.find_all('td',class_=['TableRecords_OddLine','TableRecords_EvenLine'])
            links=[]
            links = [td.find('a')['href'] for td in tds if td.find('a')]
            print(links)
            lobbyist_scrape(links)
            browser.visit(main_url)
            if navigation_has_loaded():
                try:
                    next_button = browser.find_by_css('.ListNavigation_Next')
                    next_button.click()
                    time.sleep(random.uniform(1, 1.1))
                except Exception as e: #ends loop on last page
                    print(f"Error or end of pages: {e}")
                    break
            else:
                print('bruh 2')
                break
        else:
            print('bruh')
            break


In [73]:
full_scrape()

['Lobbyist_Report.aspx?ReportId=35', 'Lobbyist_Report.aspx?ReportId=36', 'Lobbyist_Report.aspx?ReportId=37', 'Lobbyist_Report.aspx?ReportId=38', 'Lobbyist_Report.aspx?ReportId=39', 'Lobbyist_Report.aspx?ReportId=40', 'Lobbyist_Report.aspx?ReportId=41', 'Lobbyist_Report.aspx?ReportId=42', 'Lobbyist_Report.aspx?ReportId=43', 'Lobbyist_Report.aspx?ReportId=44', 'Lobbyist_Report.aspx?ReportId=45', 'Lobbyist_Report.aspx?ReportId=46', 'Lobbyist_Report.aspx?ReportId=47', 'Lobbyist_Report.aspx?ReportId=49', 'Lobbyist_Report.aspx?ReportId=50']
Skipped row with 7 columns: ['', 'Organization', 'Name', 'Address', 'Has Activity', 'Amount', '']
Skipped row with 7 columns: ['', 'AT&T', 'AT&T', '430 Bush St., 5th Floor', 'No', 'Not Stated', '']
Skipped row with 7 columns: ['', 'Ellwood Commercial Real Estate', 'Patrick Ellwood', '1354 Piedmont Ave.', 'No', 'Not Stated', '']
Skipped row with 7 columns: ['', 'Falck Ambulance Northern California', 'Carolina Snypes', '2190 S. McDowell', 'No', 'Not Stated'

In [77]:
contact_df=pd.DataFrame(records[1:],columns=records[0])
#contact_df.drop_duplicates(inplace=True, ignore_index=False)
contact_df=contact_df[['Client','Municipal Decision','Subject','Position','Narrative','City Official']]
contact_df=contact_df[contact_df.Client != 'Client']
contact_df=contact_df.reset_index(drop=True)
contact_df.to_csv('contact_df.csv')